### Intent-classifier:
Input:
 - Context and User input

Categories: 
- Valid answer
- Valid question
- Request for help
- Request for a hint
- Expression of confusion
- Boredom
- Insult
- Greeting
- Misunderstanding
- Clarification request

Output:
- Correct intent


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq




# Define the prompt template
template = """
You are an intent classifier. Given the input text, classify the intent into one of the following categories: No need of an explaination. 
- Valid answer : The user has provided a correct or appropriate response to a question or topic.
- Valid question : The user has asked a clear and relevant question related to the context or discussion.
- Irrelevent input : Using is providing a not relevent answer or asking a out-of-context question.
- Boredom : The user is expressing a lack of interest, excitement, or engagement with the current topic or situation.
- Insult : The user is using offensive or derogatory language directed at someone or something, showing disrespect.
- Greeting : The user is initiating a conversation or interaction with a friendly or polite salutation, such as "Hello" or "Good morning."
- Clarification request : The user is asking for more information or a clearer explanation to better understand something they are confused about.

Context: {context}
User input: {user_input}

Based on the above, output ONLY the final intent. 
"""

# Initialize the prompt template
prompt = PromptTemplate(
    input_variables=["context", "user_input"],  # These are the variables the template expects
    template=template,
)


In [3]:
groq_model = ChatGroq(
    temperature=0.3,
    model="llama3-70b-8192",
)

In [4]:
# Example context and user input
context = "The user has been working on an intent classifier for chatbot interactions."
user_input = "I'm not sure what this does, can you help me?"

# Use the prompt template to generate the final prompt
final_prompt = prompt.format(context=context, user_input=user_input)




In [5]:
response = groq_model.invoke(final_prompt)
response.content

'Clarification request'

In [6]:
def get_intent(context: str, user_input: str) -> str:
    context = context
    user_input = user_input

    final_prompt = prompt.format(context=context, user_input=user_input)

    response = groq_model.invoke(final_prompt)

    return response.content

In [7]:
print(get_intent(
    context="James ordered a pizza. He ate 1/3 of the Pizza and his sister, Sandra ate 1/4 of the rest of the pizza. How much Pizza left",
    user_input="Hello how are you"
))

Greeting


## using Routing 

In [8]:
from langchain.agents import AgentExecutor, create_react_agent

from langchain_community.tools.tavily_search import TavilySearchResults

context="James ordered a pizza. He ate 1/3 of the Pizza and his sister, Sandra ate 1/4 of the rest of the pizza. How much Pizza left"
user_input="1/2"
tools = [TavilySearchResults(max_results=1)]


final_prompt_template = PromptTemplate(
    input_variables=["context", "user_input"],  # Define the variables expected
    template=template  # This is your prompt string template
)

In [9]:
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

# Load environment variables
load_dotenv()

# Define the intent classifier template
template = """
You are an intent classifier. Given the input text, classify the intent into one of the following categories: No need of an explanation. 
- Valid answer : The user has provided a correct or appropriate response to a question or topic.
- Valid question : The user has asked a clear and relevant question related to the context or discussion.
- Irrelevant input : User is providing an irrelevant answer or asking an out-of-context question.
- Request for help : The user is explicitly asking for assistance or guidance to solve a problem or understand something.
- Request for a hint : The user is seeking a small clue or suggestion to help them proceed with a task, without asking for the complete solution.
- Expression of confusion : The user is indicating that they are uncertain or do not understand a concept, question, or situation.
- Boredom : The user is expressing a lack of interest, excitement, or engagement with the current topic or situation.
- Insult : The user is using offensive or derogatory language directed at someone or something, showing disrespect.
- Greeting : The user is initiating a conversation or interaction with a friendly or polite salutation, such as "Hello" or "Good morning."
- Clarification request : The user is asking for more information or a clearer explanation to better understand something they are confused about.

Context: {context}
User input: {user_input}

Based on the above, the correct intent is:
"""

# Initialize the prompt template
prompt = PromptTemplate(
    input_variables=["context", "user_input"],  # These are the variables the template expects
    template=template,
)

# Initialize the language model (Groq model)
groq_model = ChatGroq(
    temperature=0.3,
    model="llama3-70b-8192",
)

# Example context and user input
context = "James ordered a pizza. He ate 1/3 of the Pizza and his sister, Sandra ate 1/4 of the rest of the pizza. How much Pizza left"
user_input = "1/2"

# Create a TavilySearchResults tool as an example
tools = [TavilySearchResults(max_results=1)]

# Create the final prompt template for the agent
final_prompt_template = PromptTemplate(
    input_variables=["context", "user_input"],  # Define the variables you expect in the prompt
    template=template  # This is your prompt string
)

# Create the ReAct agent using the groq_model, tools, and final prompt template
agent = create_react_agent(groq_model, tools, final_prompt_template)

# Create an agent executor that will process the input using the tools and agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Invoke the agent with context and user input
response = agent_executor.invoke({"context": context, "user_input": user_input})

# Print the output response
print(response)


ValueError: Prompt missing required variables: {'tools', 'tool_names', 'agent_scratchpad'}

In [ ]:

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:

agent_executor.invoke()
